# Agents

You can pass a Runnable into an agent.

In [1]:
from langchain.agents import XMLAgent, tool, AgentExecutor
from langchain.chat_models import ChatAnthropic

In [2]:
model = ChatAnthropic(model="claude-2")

In [3]:
@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

In [4]:
tool_list = [search]

In [5]:
# Get prompt to use
prompt = XMLAgent.get_default_prompt()

In [6]:
# Logic for going from intermediate steps to a string to pass into model
# This is pretty tied to the prompt
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log


# Logic for converting tools to string to go in prompt
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

In [7]:
agent = (
    {
        "question": lambda x: x["question"],
        "intermediate_steps": lambda x: convert_intermediate_steps(x["intermediate_steps"])
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgent.get_default_output_parser()
)

In [8]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [10]:
agent_executor.invoke({"question": "whats the weather in New york?"})



> Entering new AgentExecutor chain...
 <tool>search</tool>
<tool_input>weather in new york32 degrees

<final_answer>The weather in new york is 32 degrees

> Finished chain.


{'question': 'whats the weather in New york?',
 'output': 'The weather in new york is 32 degrees'}